In [2]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.spatial import Delaunay
from scipy.spatial import Voronoi
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN, k_means
from sklearn.naive_bayes import GaussianNB
import itertools

import pickle
import time
import seaborn as sn
import matplotlib.pyplot as plt
from PreprocessFcns import * 

%matplotlib inline

In [3]:
with open('MDS-UPDRS Data - Features.pkl', 'rb') as handle:
    Data = pickle.load(handle)

In [4]:
Scores = pd.read_csv('//FS2.smpp.local\\RTO\\CIS-PD Study\\MJFF Curation\\Finalized Dataset\\Metadata Tables\\Table3.csv')
rest_tremor_dict = {'dorsal_hand_left': '3.17 Left Upper Extremity', 'flexor_digitorum_left': '3.17 Left Upper Extremity', 
                    'dorsal_hand_right': '3.17 Right Upper Extremity', 'flexor_digitorum_right': '3.17 Right Upper Extremity',
                    'anterior_thigh_left': '3.17 Left Lower Extremity', 'distal_lateral_shank_left': '3.17 Left Lower Extremity',
                    'anterior_thigh_right': '3.17 Right Lower Extremity', 'distal_lateral_shank_right': '3.17 Right Lower Extremity'}
score_dict = {'MDS-UPDRS #1: Finger Tapping - Right': '3.4 Right Hand',
              'MDS-UPDRS #1: Finger Tapping - Left': '3.4 Left Hand',
              'MDS-UPDRS #2: Hand Movements - Right': '3.5 Right Hand',
              'MDS-UPDRS #2: Hand Movements - Left': '3.5 Left Hand',
              'MDS-UPDRS #3: Pronation-Supination - Right': '3.6 Right Hand',
              'MDS-UPDRS #3: Pronation-Supination - Left': '3.6 Left Hand',
              'MDS-UPDRS #4: Toe Tapping - Right': '3.7 Right Foot',
              'MDS-UPDRS #4: Toe Tapping - Left': '3.7 Left Foot',
              'MDS-UPDRS #5: Leg Agility - Right': '3.8 Right Leg',
              'MDS-UPDRS #5: Leg Agility - Left': '3.8 Left Leg',
              'MDS-UPDRS #6: Arising from Chair': '3.9',
              'MDS-UPDRS #7: Gait': '3.10',
              'MDS-UPDRS #8: Postural Stability': '3.12',
              'MDS-UPDRS #9: Postural Hand Tremor - Right': '3.15 Right Hand',
              'MDS-UPDRS #9: Postural Hand Tremor - Left': '3.15 Left Hand',
              'MDS-UPDRS #10: Kinetic Hand Tremor - Right': '3.16 Right Hand',
              'MDS-UPDRS #10: Kinetic Hand Tremor - Left': '3.16 Left Hand',
              'MDS-UPDRS #11: Rest Tremor': rest_tremor_dict}
v_dict = {0: '2 Weeks: Time 0', 1: '2 Weeks: Time 60', 2: '1 Month'}

In [ ]:
Scores

In [ ]:
Data[1004]['MDS-UPDRS #1: Finger Tapping - Left'][0]['flexor_digitorum_left']['accel_features'].columns

In [208]:
def makeData(t, sensor, mode, task_LR=False, sens_LR=False):
    
    features = pd.DataFrame(columns=['RMSX', 'RMSY', 'RMSZ', 'rangeX', 'rangeY', 'rangeZ', 'meanX', 'meanY',
                                     'meanZ', 'varX', 'varY', 'varZ', 'skewX', 'skewY', 'skewZ', 'kurtX',
                                     'kurtY', 'kurtZ', 'xcor_peakXY', 'xcorr_peakXZ', 'xcorr_peakYZ',
                                     'xcorr_lagXY', 'xcorr_lagXZ', 'xcorr_lagYZ', 'Dom_freq', 'Pdom_rel',
                                     'PSD_mean', 'PSD_std', 'PSD_skew', 'PSD_kur', 'jerk_mean', 'jerk_std',
                                     'jerk_skew', 'jerk_kur', 'Sen_X', 'Sen_Y', 'Sen_Z', 'RMS_mag',
                                     'range_mag', 'mean_mag', 'var_mag', 'skew_mag', 'kurt_mag', 'Sen_mag'])

    # dropFeat = ['xcor_peakXY', 'xcorr_peakXZ', 'xcorr_peakYZ', 'xcorr_lagXY', 'xcorr_lagXZ', 'xcorr_lagYZ',
    #             'Sen_X', 'Sen_Y', 'Sen_Z', 'Sen_mag']

    dropFeat = ['RMSX', 'RMSY', 'RMSZ', 'rangeX', 'rangeY', 'rangeZ', 'meanX', 'meanY',
                                     'meanZ', 'varX', 'varY', 'varZ', 'skewX', 'skewY', 'skewZ', 'kurtX',
                                     'kurtY', 'kurtZ', 'xcor_peakXY', 'xcorr_peakXZ', 'xcorr_peakYZ',
                                     'xcorr_lagXY', 'xcorr_lagXZ', 'xcorr_lagYZ', 'Dom_freq', 'Pdom_rel',
                                     'PSD_mean', 'PSD_std', 'PSD_skew', 'PSD_kur', 'jerk_mean', 'jerk_std',
                                     'jerk_skew', 'jerk_kur', 'Sen_X', 'Sen_Y', 'Sen_Z']


    for subj in Data.keys():

        if task_LR:
            tasks = [task + ' - Left', task + ' - Right']
        else:
            tasks = [task]

        for t in tasks:

            for v in Data[subj][t].keys():

                if sens_LR:
                    sensors = [sensor + 'left', sensor + 'right']
                else:
                    sensors = [sensor]

                for s in sensors:
                    if (s[-3:]!=t[-3:]) & sens_LR:
                        continue

                    temp_feat = Data[subj][t][v][s][mode+'_features']

                    scoreQ = score_dict[t]
                    if type(scoreQ)==dict:
                        scoreQ=scoreQ[s]

                    score = Scores[(Scores.Visit==v_dict[v]) & (Scores['Subject ID']==subj)][scoreQ].values

                    temp_feat['Score'] = score[0]
                    temp_feat['Subj'] = subj
                    features = pd.concat([features,temp_feat],sort=True)

    features.drop(columns=dropFeat, inplace=True)
    
    return features

In [219]:
def prepFeatures(features):
    
    nan_inds = ~features.Score.isnull()
    Subjs = features.Subj[nan_inds]
    Y = features.Score[nan_inds]>0
    X = features.drop(columns=['Score', 'Subj'])[nan_inds]
    
    scaler = StandardScaler()
    pca = PCA()
    X_pca = pca.fit_transform(scaler.fit_transform(X.values))
    
    return X_pca, Y, Subjs

In [200]:
for val in features.Score.unique():
    print(val,sum(features.Score==val))

2.0 20
1.0 54
0.0 26
3.0 5


In [203]:
pca.explained_variance_ratio_

array([0.37526512, 0.20700537, 0.15935238, 0.10605156, 0.06776371,
       0.04752703, 0.03516961, 0.00186521], dtype=float32)

In [204]:
for i in range(len(pca.explained_variance_ratio_)):
    print(sum(pca.explained_variance_ratio_[0:i+1]))

0.37526512145996094
0.5822704881429672
0.7416228652000427
0.8476744219660759
0.9154381304979324
0.9629651568830013
0.9981347694993019
0.9999999782303348


In [205]:
def VorEntropy(X_pca,Y,d):
    t1 = time.time()
    Vor = Voronoi(X_pca[:,:d])
    t2 = time.time()

    score_list = sorted(Y.unique())
    class_counts = np.zeros((len(X_pca),len(score_list)))

    for p in Vor.ridge_points:
        class_counts[p[0],int(Y.values[p[1]])] += 1
        class_counts[p[1],int(Y.values[p[0]])] += 1

    totals = np.sum(class_counts,axis=1)
    class_probs = class_counts/np.vstack([totals]*len(score_list)).transpose()

    E = 0
    E0 = 0
    p_tot = 0
    p0_tot = 0
    probs = np.zeros((len(score_list),len(score_list)))
    probs0 = np.zeros((len(score_list),len(score_list)))
    for i,j in itertools.product(score_list,score_list):
        i = int(i); j = int(j)
        
        offset=0
        if i==j:
            offset=1
        
        s1_inds = Y.values==i
        p = sum(s1_inds)/len(Y) * np.mean(class_probs[s1_inds,int(j)])
        p_tot += p
        probs[i,j] = p
        E += p * sp.log2(p)

        p0 = sum(s1_inds)/len(Y) * (sum(Y.values==j)-offset)/(len(Y)-1)
        p0_tot += p0
        probs0[i,j] = p0
        E0 += p0 * sp.log2(p0)

    E = -1 * E

    print(d, t2-t1, E+E0, p_tot, p0_tot)
    return probs, probs0

In [220]:
task = 'MDS-UPDRS #1: Finger Tapping'
task_LR = True
sensor = 'sacrum'
sens_LR = False
mode = 'accel'

F = makeData(task,sensor,mode,task_LR,sens_LR)
X, Y, Subjs = prepFeatures(F)
for d in range(2,9):
    VorEntropy(X,Y,d)

2 0.0020003318786621094 0.015962576246171478 1.0 1.0
3 0.004000425338745117 -0.019617732260883036 1.0 1.0
4 0.018002033233642578 0.0013707822514950063 1.0 1.0
5 0.06500649452209473 -0.00457491176300362 1.0 1.0
6 0.29202938079833984 0.017287005501742003 1.0000000000000002 1.0
7 1.7141714096069336 0.025398255907400413 1.0 1.0
8 10.306030511856079 0.005825404558423175 1.0 1.0


In [221]:
task = 'MDS-UPDRS #1: Finger Tapping'
task_LR = True
sensor = 'dorsal_hand_'
sens_LR = True
mode = 'accel'

F = makeData(task,sensor,mode,task_LR,sens_LR)
X, Y, Subjs = prepFeatures(F)
for d in range(2,9):
    VorEntropy(X,Y,d)

2 0.002000093460083008 -3.275049834683408e-05 0.9999999999999999 1.0
3 0.005000591278076172 -0.008082850320906942 1.0 1.0
4 0.015001773834228516 0.013553260861145722 1.0 1.0
5 0.062006473541259766 0.0007889060690338123 1.0 1.0
6 0.3340332508087158 0.0038372825516055364 0.9999999999999999 1.0
7 1.8671867847442627 0.01145628258924325 1.0 1.0
8 10.002000093460083 0.0041116224849711 1.0 1.0


In [213]:
F_hand = F

In [215]:
F_sacrum = F

In [216]:
F_hand.head()

,PSD_8-12,RMS_mag,Score,Sen_mag,Subj,kurt_mag,mean_mag,range_mag,skew_mag,var_mag
0,0.028796,0.085369,2.0,1.270035,1004.0,0.197789,1.035890,1.391516,-0.409724,0.266707
1,0.002645,0.087249,2.0,1.090738,1004.0,1.342904,1.065863,1.391516,-0.735872,0.226880
2,0.005526,0.086135,2.0,1.225364,1004.0,0.871018,1.057978,1.229380,-0.463050,0.195119
3,0.005060,0.092345,2.0,0.934309,1004.0,4.523701,1.058965,2.379982,1.314007,0.339243
0,0.013628,0.082799,1.0,1.258242,1004.0,0.459388,1.020222,0.959588,-0.828039,0.188370


In [217]:
F_sacrum.head()

,PSD_8-12,RMS_mag,Score,Sen_mag,Subj,kurt_mag,mean_mag,range_mag,skew_mag,var_mag
0,1.822258e-06,0.079462,2.0,1.266948,1004.0,0.998713,0.995651,0.019902,-0.957411,0.003614
1,2.877973e-07,0.079699,2.0,1.509908,1004.0,0.181843,0.995437,0.014534,-0.656655,0.002803
2,5.008408e-07,0.079682,2.0,1.563976,1004.0,-0.042303,0.995222,0.016445,-0.245361,0.002927
3,1.804929e-06,0.082687,2.0,1.340485,1004.0,0.567933,0.995678,0.023199,0.381480,0.004170
0,8.155657e-07,0.079432,1.0,1.216395,1004.0,-0.528861,0.995273,0.013370,-0.269866,0.003084


In [61]:
for d in range(2,9):
    VorEntropy(X_pca,Y,d)

2 -0.003000020980834961 -0.13513270508780106 0.9999999999999999
3 -0.003999948501586914 -0.02061164867186216 1.0
4 -0.01699995994567871 -0.030481757647542462 1.0
5 -0.059999942779541016 0.019628132834463408 1.0
6 -0.3260002136230469 0.019240595779822733 0.9999999999999999
7 -1.696000099182129 0.022408925093042154 1.0
8 -9.506999969482422 0.012755072315110194 1.0


In [78]:
for d in range(2,9):
    VorEntropy(X_pca,Y,d)

2 0.003000020980834961 -0.0699591163139548 1.0 0.9999999999999999
3 0.004000186920166016 -0.002809103660739165 1.0 0.9999999999999999
4 0.01399993896484375 0.008738189368412197 0.9999999999999999 0.9999999999999999
5 0.05799984931945801 0.023056513182063476 0.9999999999999998 0.9999999999999999
6 0.29799985885620117 0.018047246393526617 1.0 0.9999999999999999
7 1.8559999465942383 0.00455346358701858 0.9999999999999998 0.9999999999999999
8 10.049000263214111 0.0016340478501488676 0.9999999999999998 0.9999999999999999


In [194]:
p,p0 = VorEntropy(X_pca,Y,2)

2 0.0020003318786621094 -3.275049834683408e-05 0.9999999999999999 1.0


In [195]:
p

array([[0.10142548, 0.14619357],
       [0.16210884, 0.59027211]])

In [192]:
p0

array([[0.05952381, 0.18809524],
       [0.18809524, 0.56428571]])

In [177]:
print(p*sp.log2(p))

[[-0.52444056 -0.45926296]
 [-0.47713877 -0.50603952]]


In [178]:
print(p0*sp.log2(p0))

[[-0.5 -0.5]
 [-0.5 -0.5]]


In [168]:
print(p1*sp.log2(p1))

[[-0.01435457 -0.06643856]
 [-0.06643856 -0.01435457]]


In [167]:
p1 = np.array([[.99,.01],[.01,.99]])

In [31]:
E+E0

-0.10642928577204636

In [ ]:
t = time.time()
Del = Delaunay(X_pca[:,:8])
print(time.time()-t)

In [ ]:
t = time.time()
Del = Delaunay(X_pca[:,:5])
print(time.time()-t)

score_list = sorted(Y.unique())
class_probs = np.zeros((len(X_pca),len(score_list)))

for i in range(len(X_pca)):
    neighbors = []
    for r in Del.simplices:
        if i in r:
            neighbors += [j for j in r if (j!=i) and (j not in neighbors)]
            
    scores = Y.values[neighbors]
    for s in range(len(score_list)):
        class_probs[i,s] = sum(scores==score_list[s])/len(scores)
        
E = 0
E0 = 0
for i,j in itertools.product(score_list,score_list):
    s1_inds = Y.values==i
    p = sum(s1_inds)/len(Y) * np.mean(class_probs[s1_inds,int(j)])
    print(i,j,p)
    E += p * sp.log2(p)
    
    p0 = sum(s1_inds)/len(Y) * sum(Y.values==j)/len(Y)
    E0 += p0 * sp.log2(p0)
    
E = -1 * E

print(E)

In [ ]:
E+E0

In [ ]:
len(Del.simplices)

In [ ]:
Del.neigh

In [ ]:
clust = DBSCAN(eps=4,min_samples=3)
clust.fit(X_pca[:,0:22])
print('Total', sum(Y==0)/len(Y), sum(Y==1)/len(Y), sum(Y==2)/len(Y), sum(Y==3)/len(Y), sum(Y==4)/len(Y))
for c in np.unique(clust.labels_):
    c_ind = clust.labels_==c
    c_tot = sum(c_ind)
    c_Y = Y[c_ind]
    print(c, sum(c_Y==0)/c_tot, sum(c_Y==1)/c_tot, sum(c_Y==2)/c_tot, sum(c_Y==3)/c_tot, sum(c_Y==4)/c_tot)

In [ ]:
clust = k_means(X_pca[:,0:10],n_clusters=5)
print('Total', sum(Y==0)/len(Y), sum(Y==1)/len(Y), sum(Y==2)/len(Y), sum(Y==3)/len(Y), sum(Y==4)/len(Y))
for c in np.unique(clust[1]):
    c_ind = clust[1]==c
    c_tot = sum(c_ind)
    c_Y = Y[c_ind]
    print(c, sum(c_Y==0)/c_tot, sum(c_Y==1)/c_tot, sum(c_Y==2)/c_tot, sum(c_Y==3)/c_tot, sum(c_Y==4)/c_tot)


In [ ]:
for s in [[0],[1,2,3,4]]:
    s_ind = [v in s for v in Y.values]
    plt.scatter(X_pca[s_ind,0],X_pca[s_ind,1])

In [ ]:
for s in Y.unique():
    s_ind = Y.values==s
    plt.scatter(X_pca[s_ind,0],X_pca[s_ind,1])

In [ ]:
X_pca[:,0:2]

In [ ]:
norm = features['PSD_8-12']/features.PSD_mean

In [ ]:
plt.scatter(features['PSD_8-12'],features.Score==0)

In [ ]:
np.corrcoef(features['PSD_8-12'][nan_inds],(features.Score==0)[nan_inds])

In [ ]:
NB = GaussianNB()

In [ ]:
X[nan_inds]

In [ ]:
NB.fit(X[nan_inds],Y[nan_inds])

In [ ]:
sum(NB.predict(X[nan_inds])==Y[nan_inds])/sum(nan_inds)

In [ ]:
gen_clips_mc10(Data[1004]['MDS-UPDRS #1: Finger Tapping - Left'][0]['flexor_digitorum_left']['accel'],2500)

In [ ]:
feature_extraction(clip_data=gen_clips_mc10(Data[1004]['MDS-UPDRS #1: Finger Tapping - Left'][0]['flexor_digitorum_left']['accel'],2500))

In [ ]:
Delaunay(X_pca[:,:2])

In [ ]:
Del = Delaunay(X_pca[:,:2])

In [ ]:
Del.simplices

In [ ]:
neighbors

In [ ]:
class_probs

In [ ]:
E

In [ ]:
class_probs

In [ ]:
class_probs[Y.values==0,:]